In [284]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [285]:
wp_filetypes = 'https://en.wikipedia.org/wiki/List_of_file_formats'

In [286]:
page = requests.get(wp_filetypes)

In [287]:
soup = BeautifulSoup(page.text,'html.parser')

In [318]:
#idenfity the heading tags
#create dict to track starting line of each heading tag
heading_tags = ["h1", "h2", "h3","h4"]
heading_pos = [i.sourceline for i in soup.find_all(heading_tags)]
heading_text = [i.text for i in soup.find_all(heading_tags)]

#identify the content that we are interested in and the related links
#track the line positions of each

#text and position of content
content_pos = [i.sourceline for i in soup.find_all('li')]
content_text = [i.text for i in soup.find_all('li')]

#links
href_pos = [i.sourceline for i in soup.find_all('a',href=True)]
href_link = [i['href'] for i in soup.find_all('a',href=True)]


In [330]:
def get_tag_pos(poslist,contentlist):
    #construct a dict to hold the starting and ending line of each element you are processing
    #input: two lists of same length
    #output: dict of tuples with start and end pos
    poses = {}

    for item in range(len(contentlist)):
        begin = poslist[item]
        
        #create tuple with beginning and ending indexes of each heading
        if item == len(poslist)-1:
            end = content_pos[-1] #if end of heading list take last content tag pos
        else:
            end = poslist[item+1]-1  #next line minus 1
        
        poses[contentlist[item]] = (begin,end)

    return poses

In [332]:
heading_scope = get_tag_pos(heading_pos,heading_text)

In [333]:
content_scope = get_tag_pos(content_pos,content_text)

In [336]:
links_scope = get_tag_pos(href_pos,href_link)

In [338]:
def connect_links(content,links):
    #create dict to connect the text from list content 
    #to relevant tags
    #input: two dicts
    #output: nested dict
    content_links = {}

    for text in content:
        
        #nest dict holds the position of the content and the link
        temp = {}
        temp['pos'] = content[text]
        temp['link'] = []
                    
        for link in links:
            if content[text][0] <= links[link][0] <= content[text][1]:
                temp['link'].append(link)

        content_links[text] = temp
    
    return content_links

In [340]:
files_links = connect_links(content_scope,links_scope)